# Finding Text Bearing Surfaces

## Configuring CITE libraries for almond kernel

First, we'll make a bintray repository with CITE libraries available to your almond kernel.

In [1]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [3]:
import $ivy.`edu.holycross.shot::ohco2:10.18.2`
import $ivy.`edu.holycross.shot.cite::xcite:4.1.1`
import $ivy.`edu.holycross.shot::scm:7.2.0`
import $ivy.`edu.holycross.shot::dse:6.0.4`
import $ivy.`edu.holycross.shot::citebinaryimage:3.1.1`
import $ivy.`edu.holycross.shot::citeobj:7.3.4`
import $ivy.`edu.holycross.shot::citerelations:2.5.2`
import $ivy.`edu.holycross.shot::cex:6.3.3`


import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                              

import $ivy.$                              

import $ivy.$                                          

import $ivy.$                                  

import $ivy.$                                        

import $ivy.$                              


## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [10]:
// Import some CITE libraries
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.scm._
import edu.holycross.shot.citeobj._
import edu.holycross.shot.citerelation._
import edu.holycross.shot.dse._
import edu.holycross.shot.citebinaryimage._
import edu.holycross.shot.ohco2._

import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import edu.holycross.shot.cite._

import edu.holycross.shot.ohco2._

import edu.holycross.shot.scm._

import edu.holycross.shot.citeobj._

import edu.holycross.shot.citerelation._

import edu.holycross.shot.dse._

import edu.holycross.shot.citebinaryimage._

import edu.holycross.shot.ohco2._


import almond.display.UpdatableDisplay

import almond.interpreter.api.DisplayData.ContentType

import almond.interpreter.api.{DisplayData, OutputHandler}

## Load a CITE Library

In [11]:
val filePath = s"https://raw.githubusercontent.com/Eumaeus/fuCiteDX/master/hmt/hmt_january_2020.cex"
val lib: CiteLibrary = CiteLibrarySource.fromUrl(filePath)

2020-01-11 16:21:34.241-0500  info [CiteLibrary] Building text repo from cex ...  - (CiteLibrary.scala:160)
2020-01-11 16:21:35.259-0500  info [CiteLibrary] Building collection repo from cex ...  - (CiteLibrary.scala:163)
2020-01-11 16:21:47.968-0500  info [CiteLibrary] Building relations from cex ...  - (CiteLibrary.scala:166)
2020-01-11 16:21:48.996-0500  info [CiteLibrary] All library components built.  - (CiteLibrary.scala:168)


filePath: String = "https://raw.githubusercontent.com/Eumaeus/fuCiteDX/master/hmt/hmt_january_2020.cex"
lib: CiteLibrary = CiteLibrary(
  "Homer Multitext project, release cwb_test",
  Cite2Urn("urn:cite2:hmt:publications.cex.cwb_test:all"),
  "Creative Commons Attribution, Non-Commercial 4.0 License <https://creativecommons.org/licenses/by-nc/4.0/>.",
  Vector(
    CiteNamespace("hmt", http://www.homermultitext.org/citens/hmt),
    CiteNamespace("greekLit", http://chs.harvard.edu/ctsns/greekLit)
  ),
  Some(
    TextRepository(
      Corpus(
        Vector(
          CitableNode(
            CtsUrn("urn:cts:greekLit:tlg5026.msA.hmt:1.1.lemma"),
            "\u03bc\u1fc6\u03bd\u03b9\u03bd \u1f04\u03b5\u03b9\u03b4\u03b5"
          ),
          CitableNode(
            CtsUrn("urn:cts:greekLit:tlg5026.msA.hmt:1.1.comment"),
            "\u03b6\u03b7\u03c4\u03bf\u1fe6\u03c3\u03b9 \u03b4\u03b9\u03b1 \u03c4\u03af \u1f00\u03c0\u1f78 \u03c4\u1fc6\u03c2 \u03bc\u03ae\u03bd\u03b9\u03b4\u03bf\u03

Get parts of the CITE Library in convenient form:

In [6]:
lazy val tr: TextRepository = lib.textRepository.get
lazy val corp: Corpus = tr.corpus
lazy val cat: Catalog = tr.catalog
lazy val colls: CiteCollectionRepository = lib.collectionRepository.get
lazy val rels: CiteRelationSet = lib.relationSet.get
lazy val myDseVec: DseVector = DseVector.fromCiteLibrary(lib)

tr: TextRepository = [lazy]
corp: Corpus = [lazy]
cat: Catalog = [lazy]
colls: CiteCollectionRepository = [lazy]
rels: CiteRelationSet = [lazy]
myDseVec: DseVector = DseVector(Vector())

In [7]:
myDseVec

res6: DseVector = DseVector(Vector())

## Get a Map of Text-Bearing Surfaces

If a CITE Library implements the "text bearing surface" model (`urn:cite2:cite:datamodels.v1:tbsmodel`), it knows this and can tell us about it.

In [6]:
val tbsUrn = Cite2Urn("urn:cite2:cite:datamodels.v1:tbsmodel")
val tbsCollections: Vector[Cite2Urn] = lib.collectionsForModel(tbsUrn)

tbsUrn: Cite2Urn = Cite2Urn("urn:cite2:cite:datamodels.v1:tbsmodel")
tbsCollections: Vector[Cite2Urn] = Vector(
  Cite2Urn("urn:cite2:hmt:msA.v1:"),
  Cite2Urn("urn:cite2:hmt:msB.v1:")
)

With this, we can get a map of Text Bearing Surfaces, keyed by collection.

In [7]:
val tbsMap: Map[Cite2Urn, Vector[Cite2Urn]] = {
    tbsCollections.map( tc => {
        val key = tc
        val vec = (colls ~~ tc).map(_.urn)
        (key -> vec)
    }).toMap
}

tbsMap: Map[Cite2Urn, Vector[Cite2Urn]] = Map(
  Cite2Urn("urn:cite2:hmt:msA.v1:") -> Vector(
    Cite2Urn("urn:cite2:hmt:msA.v1:insidefrontcover"),
    Cite2Urn("urn:cite2:hmt:msA.v1:ir"),
    Cite2Urn("urn:cite2:hmt:msA.v1:iv"),
    Cite2Urn("urn:cite2:hmt:msA.v1:1r"),
    Cite2Urn("urn:cite2:hmt:msA.v1:1v"),
    Cite2Urn("urn:cite2:hmt:msA.v1:2r"),
    Cite2Urn("urn:cite2:hmt:msA.v1:2v"),
    Cite2Urn("urn:cite2:hmt:msA.v1:3r"),
    Cite2Urn("urn:cite2:hmt:msA.v1:3v"),
    Cite2Urn("urn:cite2:hmt:msA.v1:4r"),
    Cite2Urn("urn:cite2:hmt:msA.v1:4v"),
    Cite2Urn("urn:cite2:hmt:msA.v1:5r"),
    Cite2Urn("urn:cite2:hmt:msA.v1:5v"),
    Cite2Urn("urn:cite2:hmt:msA.v1:6r"),
    Cite2Urn("urn:cite2:hmt:msA.v1:6v"),
    Cite2Urn("urn:cite2:hmt:msA.v1:7r"),
    Cite2Urn("urn:cite2:hmt:msA.v1:7v"),
    Cite2Urn("urn:cite2:hmt:msA.v1:8r"),
    Cite2Urn("urn:cite2:hmt:msA.v1:8v"),
    Cite2Urn("urn:cite2:hmt:msA.v1:9r"),
    Cite2Urn("urn:cite2:hmt:msA.v1:9v"),
    Cite2Urn("urn:cite2:hmt:msA

   ## DSE Work

In [8]:
lib.dseVector

cmd8.sc:1: value dseVector is not a member of edu.holycross.shot.scm.CiteLibrary
val res8 = lib.dseVector
               ^Compilation Failed

: 

In [9]:
DseVector.fromCiteLibrary(lib)

res8: DseVector = DseVector(
  Vector(
    DsePassage(
      Cite2Urn("urn:cite2:hmt:va_dse.v1:il2168"),
      "DSE record for Iliad 4.217",
      CtsUrn("urn:cts:greekLit:tlg0012.tlg001.msA:4.217"),
      Cite2Urn(
        "urn:cite2:hmt:vaimg.2017a:VA055VN_0557@0.4865,0.3644,0.3954,0.0391"
      ),
      Cite2Urn("urn:cite2:hmt:msA.v1:55v")
    ),
    DsePassage(
      Cite2Urn("urn:cite2:hmt:va_dse.v1:il11826"),
      "DSE record for Iliad 18.529",
      CtsUrn("urn:cts:greekLit:tlg0012.tlg001.msA:18.529"),
      Cite2Urn(
        "urn:cite2:hmt:vaimg.2017a:VA249RN_0420@0.19,0.6589,0.427,0.0331"
      ),
      Cite2Urn("urn:cite2:hmt:msA.v1:249r")
    ),
    DsePassage(
      Cite2Urn("urn:cite2:hmt:va_dse.v1:il6005"),
      "DSE record for Iliad 10.337",
      CtsUrn("urn:cts:greekLit:tlg0012.tlg001.msA:10.337"),
      Cite2Urn(
        "urn:cite2:hmt:vaimg.2017a:VA132VN_0635@0.485,0.4313,0.388,0.0278"
      ),
      Cite2Urn("urn:cite2:hmt:msA.v1:132v")
    ),
    DsePassage(
    

In [12]:
val ctsu1 = CtsUrn("urn:cts:greekLit:tlg0012.tlg001:1")
val ctsu2 = CtsUrn("urn:cts:greekLit:tlg0012.tlg001:1")
val citeu1 =  Cite2Urn("urn:cite2:hmt:va_dse.v1:il16223")
val citeu2 =  Cite2Urn("urn:cite2:hmt:va_dse.v1:il16223")

ctsu1: CtsUrn = CtsUrn("urn:cts:greekLit:tlg0012.tlg001:1")
ctsu2: CtsUrn = CtsUrn("urn:cts:greekLit:tlg0012.tlg001:1")
citeu1: Cite2Urn = Cite2Urn("urn:cite2:hmt:va_dse.v1:il16223")
citeu2: Cite2Urn = Cite2Urn("urn:cite2:hmt:va_dse.v1:il16223")

In [14]:
ctsu1.asInstanceOf[Urn] 
ctsu2.asInstanceOf[Urn] 

res13_0: Urn = CtsUrn("urn:cts:greekLit:tlg0012.tlg001:1")
res13_1: Urn = CtsUrn("urn:cts:greekLit:tlg0012.tlg001:1")

In [15]:
res13_0 == res13_1


res14: Boolean = true